## Import
Use elements of the scientific python stack

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr

# Import a client
from dask.distributed import Client

# Import intake-esm
import intake

import util

## Setup a PBSCluster
Speed up processing with DASK with 8 threads

In [ ]:
cluster = util.get_pbscluster(8)
# Setup your client
client = Client(cluster)
client

## Open a catalog

In [ ]:
cat = intake.open_esm_datastore(
    "/glade/work/marsh/intake-esm-catalogs/WACCM6-TSMLT-GEO-SAI1.json",
    ).search(
    variable=('Uzm'),
    frequency=('day_1')
)
print(cat.keys())
# cat

In [ ]:
dsets = cat.to_dataset_dict(cdf_kwargs={'chunks': {'time': 8}, 'decode_times': False})
dsets.keys()

In [ ]:
def plot_u(u, case):
    
    print(case)
    fig = u.plot.contourf(y='ilev', 
                          ylim=(100,1), 
                          yincrease=False, 
                          yscale='log',
                          figsize=(15,6), 
                          levels=np.arange(-30,31,2.5)
                         )
    fig.axes.set_ylabel('Pressure (hPa)')
    fig.axes.set_xlabel('Year')
    plt.title('Equatorial zonal mean U')

    plt.show()
    return

In [ ]:
hist_cases = [key for key in list(cat) if "SSP245" in key]
print(len(hist_cases))

In [ ]:
for case in hist_cases:
    # print(case)
    ds = dsets[case]
    ds = xr.decode_cf(ds)
    ds = ds.sortby('time')
    u0 = ds['Uzm'].isel(zlon=0).sel(lat=0.0, method='nearest').sel(time=slice("2035-01", "2052-12"))
    plot_u(u0, case)
    u0 = ds['Uzm'].isel(zlon=0).sel(lat=0.0, method='nearest').sel(time=slice("2053-01", "2070-12"))
    plot_u(u0, case)

    